# Path Config & Global Imports

In [ ]:
from pathlib import Path
import os

# -----------------------
# 🔧 Project Path Config
# -----------------------
# Set project base path dynamically using the folder where the notebook is opened.
# This makes the notebook fully portable across different machines without hard-coded directories.

# Use the folder where the notebook is opened
# This means the CSV is expected to be in the same folder as the notebook, and Python finds it 
#  relative to the notebook location
BASE_PATH = Path.cwd()   # used it everywhere a file path is needed
print("Working in:", BASE_PATH)

os.environ["OMP_NUM_THREADS"] = "1"  # must be set before importing numpy/sklearn

import warnings
warnings.filterwarnings(
    "ignore",
    message="'force_all_finite' was renamed to 'ensure_all_finite'",
    category=FutureWarning
)


# Load Data and Create Trace Strings for Vectorization

In [ ]:
# ======================================================
# Vectorization & Clustering Pipeline (Annotated Version)
# ======================================================


# This script loads event-log data, constructs textual variants of cases,
# encodes them using TF-IDF and Doc2Vec, reduces dimensionality when needed,
# and runs several clustering algorithms (KMeans, SOM, HDBSCAN). Metrics and
# cluster-level summaries are also computed.

# ======================================================
# 0) Imports & Global Config
# ======================================================
# Standard library
import os
import random
from pathlib import Path

# Numerical / Data handling
import numpy as np
import pandas as pd

# Visualization
import matplotlib.pyplot as plt
from IPython.display import display

# ML / NLP
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score, calinski_harabasz_score, davies_bouldin_score

from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from minisom import MiniSom
from hdbscan import HDBSCAN

# ======================================================
# Global Random Seed (for reproducibility across modules)
# ======================================================
GLOBAL_SEED = 42
np.random.seed(GLOBAL_SEED)
random.seed(GLOBAL_SEED)
os.environ["PYTHONHASHSEED"] = str(GLOBAL_SEED)


# ======================================================
# 1) Load Data
# ======================================================
# The script expects a CSV containing an event log with:
# - case:concept:name (case IDs)
# - time:timestamp (event timestamp)
# - concept:name (activity name)
# Sampled subset is used for feasibility.
log_path = BASE_PATH / "df_sampled_100_cases.csv"

df = pd.read_csv(log_path)

df["time:timestamp"] = pd.to_datetime(df["time:timestamp"], errors="coerce")
df = df.dropna(subset=["time:timestamp"])
df = df.sort_values(["case:concept:name", "time:timestamp"]).copy()


# ======================================================
# 2) Variant Filtering
# ======================================================
# Each case is turned into a trace string: "A B C D" representing activity sequence.
# We count identical variants and group rare ones into a single "__OTHER__" bucket.
print("\n🔍 Variant frequency analysis...")

# Build the trace strings per case
seqs = (
    df.groupby("case:concept:name")["concept:name"]
    .apply(lambda s: " ".join(s.astype(str).tolist()))
    .rename("trace_str")
    .to_frame()
)

# Count frequency of each unique variant
variant_counts = (
    seqs["trace_str"]
    .value_counts()
    .rename_axis("variant")
    .reset_index(name="count")
)
variant_counts["pct"] = variant_counts["count"] / variant_counts["count"].sum()

# Keep variants with at least this frequency; others lumped
min_variant_freq = 2
common_variants = set(
    variant_counts.loc[variant_counts["count"] >= min_variant_freq, "variant"]
)

# Add a "variant" column assigning rare cases to "__OTHER__"
filtered = seqs.copy()
filtered["variant"] = filtered["trace_str"].where(
    filtered["trace_str"].isin(common_variants), "__OTHER__"
)

# Summary table for transparency
filter_stats = pd.DataFrame({
    "total_cases": [len(seqs)],
    "unique_variants_total": [variant_counts.shape[0]],
    "min_variant_freq": [min_variant_freq],
    "unique_variants_kept": [len(common_variants) + (1 if "__OTHER__" in filtered["variant"].unique() else 0)],
    "cases_in_other": [(filtered["variant"] == "__OTHER__").sum()]
})

print("\n📊 Variant filtering summary:")
display(filter_stats)



# Vectorization and Clustering

In [ ]:
# ======================================================
# 3) Vectorization Function
# ======================================================
def vectorize(sequences, method="both"):
    """
    Vectorize trace sequences into numerical embeddings.

    Parameters
    ----------
    sequences : pd.Series
        One trace string per case, e.g. 'A B C D'.
    method : {'both', 'tfidf', 'doc2vec'}
        Which embeddings to compute.

    Returns
    -------
    encoders : dict
        Mapping name -> 2D numpy array of embeddings.
        Keys will include 'TFIDF_SVD' and/or 'DOC2VEC'.
    artifacts : dict
        Any fitted objects (tfidf, svd, doc2vec_model) for reuse.
    """
    encoders = {}
    artifacts = {}

    # --- TF-IDF + SVD ---
    if method in ("both", "tfidf"):
        tfidf = TfidfVectorizer(
            token_pattern=r"[^ ]+",
            lowercase=False,
            ngram_range=(1, 3)
        )
        X_tfidf = tfidf.fit_transform(sequences)

        svd_components = max(min(10, X_tfidf.shape[1] - 1), 2)
        svd = TruncatedSVD(n_components=svd_components, random_state=GLOBAL_SEED)
        X_tfidf_svd = svd.fit_transform(X_tfidf)

        X_tfidf_std = StandardScaler().fit_transform(X_tfidf_svd)

        encoders["TFIDF_SVD"] = X_tfidf_std
        artifacts["tfidf"] = tfidf
        artifacts["svd"] = svd

    # --- Doc2Vec ---
    if method in ("both", "doc2vec"):
        tagged_docs = [
            TaggedDocument(words=trace.split(), tags=[str(i)])
            for i, trace in enumerate(sequences)
        ]
        doc2vec_model = Doc2Vec(
            vector_size=100,
            min_count=1,
            epochs=40,
            seed=GLOBAL_SEED,
            workers=1
        )
        doc2vec_model.build_vocab(tagged_docs)
        doc2vec_model.train(
            tagged_docs,
            total_examples=doc2vec_model.corpus_count,
            epochs=doc2vec_model.epochs
        )
        X_doc2vec = np.array([
            doc2vec_model.infer_vector(trace.split(), epochs=40, alpha=0.025)
            for trace in sequences
        ])
        X_doc2vec_std = StandardScaler().fit_transform(X_doc2vec)

        encoders["DOC2VEC"] = X_doc2vec_std
        artifacts["doc2vec_model"] = doc2vec_model

    print("\n✅ Embedding shapes:")
    for name, X in encoders.items():
        print(f"  - {name}: {X.shape}")

    return encoders, artifacts


# Call the vectorization function on your trace sequences
encoders, encoder_artifacts = vectorize(seqs["trace_str"], method="both")


# ======================================================
# 4) Core Clustering Helpers (as before)
# ======================================================
def evaluate_clustering(X, labels):
    """Compute three internal clustering metrics.
    If the clustering degenerates (one cluster or all unique labels), return NaN.
    """
    unique = np.unique(labels)
    if len(unique) < 2 or len(unique) == len(labels):
        return {"silhouette": np.nan, "ch": np.nan, "db": np.nan}
    return {
        "silhouette": silhouette_score(X, labels),
        "ch": calinski_harabasz_score(X, labels),
        "db": davies_bouldin_score(X, labels)
    }


# ------------------------------------------------------
# KMeans sweep
# ------------------------------------------------------
def run_kmeans_sweep(X, k_values):
    """Try multiple k values and return both all results and the best one."""
    best = None
    all_res = []

    for k in k_values:
        km = KMeans(n_clusters=k, n_init=20, random_state=GLOBAL_SEED)
        labels = km.fit_predict(X)
        metrics = evaluate_clustering(X, labels)

        result = {"k": k, "model": km, "labels": labels, **metrics}
        all_res.append(result)

        if best is None or (
            metrics["silhouette"] > best["silhouette"] or
            (
                np.isclose(metrics["silhouette"], best["silhouette"]) and
                metrics["ch"] > best["ch"]
            )
        ):
            best = result

    return best, all_res


# ------------------------------------------------------
# SOM without BMUs
# ------------------------------------------------------
def run_som_no_bmu(X, n_clusters, xdim=5, ydim=5,
                   sigma=1.0, learning_rate=0.5, iters=5000):
    """Train a SOM and cluster neuron weights using KMeans.
    Each data point is mapped to the label of its closest neuron.
    """
    # SOM expects normalized input
    X_scaled = MinMaxScaler().fit_transform(X)
    
    # Initialize SOM grid
    som = MiniSom(
        x=xdim, y=ydim,
        input_len=X_scaled.shape[1],
        sigma=sigma,
        learning_rate=learning_rate,
        random_seed=42
    )

    som.random_weights_init(X_scaled)
    som.train_random(X_scaled, iters)
    
    # Flatten neuron weight vectors
    weights = som.get_weights().reshape(-1, X_scaled.shape[1])
    
    # Cluster neurons
    km = KMeans(n_clusters=n_clusters, n_init=20, random_state=GLOBAL_SEED)
    neuron_labels = km.fit_predict(weights)
    
    # Map each data point to its nearest neuron
    dists = np.linalg.norm(
        X_scaled[:, None, :] - weights[None, :, :],
        axis=2
    )
    closest = dists.argmin(axis=1)

    return neuron_labels[closest]


# Sweep over K for SOM
# ------------------------------------------------------
def run_som_sweep(X, k_values):
    """
    Try different values of k (number of clusters) for a Self-Organizing Map (SOM).

    Parameters
    ----------
    X : np.ndarray
        The embedding matrix for all traces (e.g., TF-IDF or Doc2Vec vectors).
    k_values : iterable
        A list or range of k values to evaluate, such as range(2, 16).

    Returns
    -------
    best : dict
        The single best clustering result found across all k values.
        Contains:
            - 'k'          : the chosen number of clusters
            - 'labels'     : cluster labels for each trace
            - metrics...   : silhouette, CH, DB scores
    all_res : list of dict
        A list of the clustering results for *every* k tested.
        Useful for plotting elbow/silhouette trends.
    """

    best = None         # will store the best performing clustering configuration
    all_res = []        # will store all results for later analysis

    # Try each possible number of clusters
    for k in k_values:

        # Run SOM-based clustering for this k
        labels = run_som_no_bmu(X, n_clusters=k)

        # Compute internal cluster quality metrics
        metrics = evaluate_clustering(X, labels)

        # Package results for this k
        result = {"k": k, "labels": labels, **metrics}
        all_res.append(result)

        # Keep track of the best result so far:
        # Priority 1 → max silhouette
        # Priority 2 → max Calinski-Harabasz score (if silhouette ties)
        if best is None or (
            metrics["silhouette"] > best["silhouette"] or
            (
                np.isclose(metrics["silhouette"], best["silhouette"]) and
                metrics["ch"] > best["ch"]
            )
        ):
            best = result

    return best, all_res



# ------------------------------------------------------
# HDBSCAN
# ------------------------------------------------------
def run_hdbscan(X, min_cluster_size=5):
    """Run density-based clustering. Cluster count is determined automatically."""
    hdb = HDBSCAN(min_cluster_size=min_cluster_size,
                  min_samples=min_cluster_size,
                  metric="euclidean",
                  cluster_selection_method="eom")

    labels = hdb.fit_predict(X)
    metrics = evaluate_clustering(X, labels)

    return {
        "model": hdb,
        "labels": labels,
        "k": len(np.unique(labels[labels >= 0])),
        **metrics
    }


# ------------------------------------------------------
# Cluster summary
# ------------------------------------------------------
def cluster_summary(labels, seqs_df):
    """Compute summary statistics for each cluster: size, trace lengths,
    entropy of variants, etc."""
    tmp = seqs_df.copy()
    tmp["_cluster_tmp"] = labels

    rows = []
    for c, g in tmp.groupby("_cluster_tmp"):
        n = len(g)
        lengths = g["trace_str"].str.split().map(len)
        vc = g["variant"].value_counts(normalize=True)
        # Variant entropy approximates intra-cluster variability
        entropy = -np.sum(vc * np.log2(vc + 1e-12))

        rows.append({
            "cluster": int(c),
            "size": n,
            "pct": round(100*n/len(seqs_df), 2),
            "other_cases": (g["variant"] == "__OTHER__").sum(),
            "avg_len": float(np.mean(lengths)),
            "median_len": float(np.median(lengths)),
            "variant_entropy": float(entropy),
            "unique_variants": g["variant"].nunique()
        })
    return pd.DataFrame(rows).sort_values("size", ascending=False)


# ======================================================
# 5) Supervisor-Style Wrapper Functions
# ======================================================
def find_num_clusters(X, k_values, cluster_algo="kmeans"):
    """
    Wrapper to find the best number of clusters for a given algorithm.
    
    Parameters
    ----------
    X : np.ndarray
        Embeddings.
    k_values : iterable
        Values of k to try.
    cluster_algo : {'kmeans', 'som'}
    
    Returns
    -------
    best_result : dict
    all_results : list of dict
    """
    if cluster_algo == "kmeans":
        return run_kmeans_sweep(X, k_values)
    elif cluster_algo == "som":
        return run_som_sweep(X, k_values)
    else:
        raise ValueError(f"Unsupported cluster_algo: {cluster_algo}")

# (Note: cluster_traces is not used in automated sweep pipeline, but added based on Williams 
#        feedback)
def cluster_traces(X, num_clusters, cluster_algo="kmeans"):
    """
    -Perform clustering given vectors X and chosen number of clusters.
    -Returns cluster labels and the fitted model (if applicable).
    -Makes my clustering code re-usable on other datasets for those who don't want to run
    "sweep" functions
    -Allows for manual clustering when k is known
    """
    if cluster_algo == "kmeans":
        km = KMeans(n_clusters=num_clusters, n_init=20, random_state=GLOBAL_SEED)
        labels = km.fit_predict(X)
        return labels, km
    elif cluster_algo == "som":
        labels = run_som_no_bmu(X, n_clusters=num_clusters)
        return labels, None
    else:
        raise ValueError(f"Unsupported cluster_algo: {cluster_algo}")


def mine_from_clusters(labels, num_clusters, sequences_df):
    """
    Mine cluster-level summaries from cluster labels and sequences.
    Here we reuse the existing `cluster_summary` function.

    Parameters
    ----------
    labels : array-like
        Cluster labels for each trace.
    num_clusters : int
        Number of clusters (not strictly needed here, but kept for API consistency).
    sequences_df : pd.DataFrame
        DataFrame with at least 'trace_str' and 'variant' columns.

    Returns
    -------
    pd.DataFrame
        Cluster summary statistics.
    """
    return cluster_summary(labels, sequences_df)


# ======================================================
# 6) Run All Clustering (KMeans, SOM, HDBSCAN) using the new helpers
# ======================================================
K_RANGE = range(2, 16)

algo_results = []
cluster_summaries = {}

all_kmeans_sweeps = {}
all_som_sweeps = {}

for enc_name, X in encoders.items():
    print(f"\n🚀 Running: {enc_name}")

    # --- KMeans ---
    best_km, km_all = find_num_clusters(X, K_RANGE, cluster_algo="kmeans")
    all_kmeans_sweeps[enc_name] = km_all
    filtered[f"cluster_kmeans_{enc_name.lower()}"] = best_km["labels"]
    cluster_summaries[(enc_name, "KMeans")] = mine_from_clusters(
        best_km["labels"],
        best_km["k"],
        filtered
    )

    algo_results.append({
        "encoder": enc_name,
        "algorithm": "KMeans",
        **{k: best_km[k] for k in ["k", "silhouette", "ch", "db"]}
    })

    # --- SOM ---
    best_som, som_all = find_num_clusters(X, K_RANGE, cluster_algo="som")
    all_som_sweeps[enc_name] = som_all
    filtered[f"cluster_som_{enc_name.lower()}"] = best_som["labels"]
    cluster_summaries[(enc_name, "SOM")] = mine_from_clusters(
        best_som["labels"],
        best_som["k"],
        filtered
    )

    algo_results.append({
        "encoder": enc_name,
        "algorithm": "SOM",
        **{k: best_som[k] for k in ["k", "silhouette", "ch", "db"]}
    })

    # --- HDBSCAN (unchanged in spirit) ---
    hdb = run_hdbscan(X, min_cluster_size=2)
    filtered[f"cluster_hdbscan_{enc_name.lower()}"] = hdb["labels"]
    cluster_summaries[(enc_name, "HDBSCAN")] = mine_from_clusters(
        hdb["labels"],
        hdb["k"],
        filtered
    )

    algo_results.append({
        "encoder": enc_name,
        "algorithm": "HDBSCAN",
        **{k: hdb[k] for k in ["k", "silhouette", "ch", "db"]}
    })


# ======================================================
# 7) Algorithm Comparison Table
# ======================================================
algo_df = pd.DataFrame(algo_results)
print("\n📈 Algorithm Comparison")
display(algo_df)


# ======================================================
# 8) Silhouette Trend Plotting (unchanged)
# ======================================================
def plot_silhouette_trend_dict(results, title):
    k_vals = [r["k"] for r in results if not np.isnan(r["silhouette"])]
    sil_vals = [r["silhouette"] for r in results if not np.isnan(r["silhouette"])]

    if len(k_vals) == 0:
        print(f"No valid silhouette values for {title}.")
        return

    plt.figure(figsize=(6, 4))
    plt.plot(k_vals, sil_vals, marker='o', linewidth=2)
    plt.title(title)
    plt.xlabel("k")
    plt.ylabel("Silhouette")
    plt.ylim(0, 1)
    plt.grid(True, alpha=0.3)
    plt.show()


print("\n📈 Silhouette Trends")

print("\nTFIDF — KMeans")
plot_silhouette_trend_dict(all_kmeans_sweeps["TFIDF_SVD"], "KMeans (TFIDF)")

print("\nTFIDF — SOM")
plot_silhouette_trend_dict(all_som_sweeps["TFIDF_SVD"], "SOM (TFIDF)")

print("\nDOC2VEC — KMeans")
plot_silhouette_trend_dict(all_kmeans_sweeps["DOC2VEC"], "KMeans (DOC2VEC)")

print("\nDOC2VEC — SOM")
plot_silhouette_trend_dict(all_som_sweeps["DOC2VEC"], "SOM (DOC2VEC)")

print("\n⚙️ Skipping HDBSCAN silhouette trends (auto-detected clusters).")


# Process Discovery - Global

In [ ]:
# =============================================================
# Global Process Discovery Pipeline (Annotated Version)
# =============================================================
# This script performs *global* process discovery using PM4Py.
# It discovers a single process model for the entire log (no clustering),
# evaluates conformance (fitness + precision), visualizes the resulting
# models (Petri net / BPMN / Heuristics Net), and computes variability.


# =============================================================
# 0) Imports
# =============================================================
# PM4Py evaluation
from pm4py.algo.evaluation.precision import algorithm as precision_evaluator
from pm4py.algo.evaluation.replay_fitness.algorithm import Variants as FitnessVariants
from pm4py.algo.evaluation.precision.algorithm import Variants as PrecisionVariants
import pandas as pd
import numpy as np

# Log conversion utilities
from pm4py.objects.log.util import dataframe_utils
from pm4py.objects.conversion.log import converter as log_converter

# Fitness (token-based replay) and precision
from pm4py.algo.evaluation.replay_fitness.variants import token_replay as fitness_evaluator
from pm4py.algo.evaluation.precision import algorithm as precision_evaluator
from pm4py.objects.log.util import dataframe_utils
from pm4py.objects.conversion.log import converter as log_converter
from pm4py.algo.evaluation.replay_fitness.variants import token_replay as fitness_evaluator
from pm4py.algo.evaluation.precision import algorithm as precision_evaluator

from pm4py.visualization.heuristics_net import visualizer as hn_visualizer

# =============================================================
# 1) Configuration
# =============================================================
MINER_TYPE = "inductive"  # options: "inductive", "heuristics", "alpha"
RANDOM_STATE = 42         # deterministic sampling
EVAL_SAMPLE_SIZE = 2000   # conformance evaluation log sample size

# Conformance settings
# checks *fit* (does the model reproduce behavior?)
FITNESS_VARIANT = FitnessVariants.TOKEN_BASED
# checks *specificity* (does it avoid overgeneralizing?)
PRECISION_VARIANT = PrecisionVariants.ETCONFORMANCE_TOKEN




In [ ]:


def discover_model_for_miner(log):
    """
    Wrapper to call the correct PM4Py discovery algorithm based on MINER_TYPE.


    Returns:
    (net, im, fm, heu_net, process_tree)
    - heuristics miner returns heu_net instead
    - inductive miner returns process tree
    """
    if MINER_TYPE == "inductive":
        process_tree = inductive_miner.apply(log)
        net, im, fm = pt_converter.apply(process_tree)
        return net, im, fm, None, process_tree     # heuristics net = None

    elif MINER_TYPE == "alpha":
        net, im, fm = alpha_miner.apply(log)
        return net, im, fm, None, None     # heuristics net = None

    elif MINER_TYPE == "heuristics":
        heu_net = heuristics_miner.apply_heu(log)
        net, im, fm = hn_converter.apply(heu_net)
        return net, im, fm, heu_net, None

    else:
        raise ValueError(f"❌ Unknown MINER_TYPE: {MINER_TYPE}")

# =============================================================
# 3) Sampling Utility
# =============================================================
def maybe_sample_log(event_log, max_traces):
    """Sample a subset of traces from a log if it exceeds max_traces.

    Args:
        event_log (EventLog): PM4Py event log object.
        max_traces (int): Maximum number of traces to keep.

    Returns:
        EventLog: A sampled or original log.
    """
    if (max_traces is None) or (len(event_log) <= max_traces):
        return event_log
    idx = np.random.RandomState(RANDOM_STATE).choice(len(event_log), size=max_traces, replace=False)
    return EventLog([event_log[i] for i in sorted(idx)])

# =============================================================
# 4) Variability Measure
# =============================================================
# Variant variability reflects behavioral diversity.
def compute_variability_ratio(log):
    """
    Compute variability ratio = (# unique variants) / (# total traces).
    Lower values indicate more homogeneous clusters.

    Args:
        log (EventLog): PM4Py EventLog for the cluster.

    Returns:
        float: variability ratio
    """
    if len(log) == 0:
        return 0.0
    variant_set = set()
    for trace in log:
        seq = tuple(e["concept:name"] for e in trace)
        variant_set.add(seq)
    return len(variant_set) / len(log)

In [ ]:
# This section MUST be ran before Process discovery
from pm4py.algo.evaluation.replay_fitness import algorithm as fitness_evaluator
from pm4py.algo.evaluation.precision import algorithm as precision_evaluator

from pm4py.algo.discovery.inductive import algorithm as inductive_miner
from pm4py.objects.conversion.process_tree import converter as pt_converter

from pm4py.algo.discovery.heuristics import algorithm as heuristics_miner
from pm4py.objects.conversion.heuristics_net import converter as hn_converter

from pm4py.algo.evaluation.replay_fitness.algorithm import Variants as FitnessVariants
from pm4py.algo.evaluation.precision.algorithm import Variants as PrecisionVariants

from pm4py.visualization.petri_net import visualizer as pn_visualizer

In [ ]:
from pm4py.visualization.petri_net import visualizer as pn_visualizer
from pm4py.objects.conversion.process_tree import converter as process_tree_converter
from pm4py.visualization.bpmn import visualizer as bpmn_visualizer

# =============================================================
# 5) Global Process Discovery + Evaluation
# =============================================================
def evaluate_global_model(df, metrics_df=None):
    """
    Perform discovery → visualization → conformance evaluation for a *global* model.


    Args:
    df (pd.DataFrame): Raw event log with columns:
    - case:concept:name
    - concept:name
    - time:timestamp


    metrics_df (pd.DataFrame or None):
    – If provided, append new results.
    – If None, create the table.


    Returns:
    pd.DataFrame: Updated metrics table.
    """

    # ----------------------------------------------
    # Convert pandas → PM4Py EventLog
    # ----------------------------------------------
    params = {log_converter.Variants.TO_EVENT_LOG.value.Parameters.CASE_ID_KEY: "case:concept:name"}
    global_log = log_converter.apply(df, variant=log_converter.Variants.TO_EVENT_LOG, parameters=params)

    # ----------------------------------------------
    # Model discovery
    # ----------------------------------------------
    net, im, fm, heu_net, process_tree = discover_model_for_miner(global_log)
    
    # ----------------------------------------------
    # Visualization (Petri net, BPMN, or heuristics net)
    # ----------------------------------------------
   # --- Visualization settings ---
    if MINER_TYPE == "inductive":
        VISUALIZATION_THRESHOLD = None  # set to None = no size limit

        #from pm4py.visualization.petri_net import visualizer as pn_visualizer

        print("PN size:", len(net.places), len(net.transitions))

        if VISUALIZATION_THRESHOLD is None or len(net.transitions) < VISUALIZATION_THRESHOLD:
            #print("Rendering PN...")
            #gviz = pn_visualizer.apply(net, im, fm)
            #pn_visualizer.view(gviz)
            print("Rendering BPMN...")
            bpmn_graph = process_tree_converter.apply(
            process_tree,
            variant=process_tree_converter.Variants.TO_BPMN)
            
            gviz = bpmn_visualizer.apply(bpmn_graph)
            bpmn_visualizer.view(gviz)
        else:
            print(f"PN too large ({len(net.transitions)} transitions) — skipping visualization")
            
    elif MINER_TYPE == "heuristics":
        # heuristics nets need their own visualizer
        
        print("Rendering Heuristics Net...")
        gviz = hn_visualizer.apply(heu_net)
        hn_visualizer.view(gviz)
    # --- END ADDED BLOCK ---
    
    # ----------------------------------------------
    # Conformance checking
    # ----------------------------------------------
    # maybe sample
    eval_log = maybe_sample_log(global_log, EVAL_SAMPLE_SIZE)

    # conformance eval
    fit_res  = fitness_evaluator.apply(eval_log, net, im, fm, variant=FITNESS_VARIANT)
    fitness  = fit_res.get("average_trace_fitness", fit_res.get("perc_fit_traces", np.nan))
    precision = precision_evaluator.apply(eval_log, net, im, fm, variant=PRECISION_VARIANT)
    fscore    = 2 * (precision * fitness) / (precision + fitness) if (precision + fitness) > 0 else 0.0

    # variability
    global_variability = compute_variability_ratio(global_log)

    # ----------------------------------------------
    # Build result row
    # ----------------------------------------------
    row_dict = {
        "Method": "Global",
        "Cluster": "Global",
        "NumTraces": len(global_log),
        "Precision": float(precision),
        "Fitness": float(fitness),
        "FScore": float(fscore),
        "VariabilityRatio": float(global_variability)
    }

    # Initialize or append
    if (metrics_df is None) or (len(metrics_df) == 0):
        metrics_df = pd.DataFrame([row_dict])
    else:
        # subsequent calls
        metrics_df = pd.concat([metrics_df, pd.DataFrame([row_dict])], ignore_index=True)

    print(
        f"🌐 Global Model ({MINER_TYPE}) → "
        f"Precision: {precision:.3f}, Fitness: {fitness:.3f}, "
        f"F-score: {fscore:.3f}, Variability Ratio: {global_variability:.3f}"
    )

    return metrics_df



## When MINER_TYPE = "heuristics" 

In [ ]:

metrics_df = evaluate_global_model(df)
display(metrics_df)

## When MINER_TYPE = "inductive" 

In [ ]:

metrics_df = evaluate_global_model(df)
display(metrics_df)

# Process Discovery - Per Cluster

## Inductive Miner

In [ ]:
# =============================================================
# Per‑Cluster Process Discovery & Evaluation (Annotated Version)
# =============================================================
# This script mirrors the global process discovery workflow but applied
# separately to each cluster. For every cluster, we:
# 1. Extract all traces belonging to that cluster
# 2. Convert them to an EventLog
# 3. Discover a process model (IMf version of Inductive Miner)
# 4. Optionally visualize the model (BPMN)
# 5. Compute conformance (precision, fitness, F-score)
# 6. Compute variability ratio
# 7. Append results to a cluster‑level metrics table


# =============================================================
# 0) Discovery Function — IMf Variant
# =============================================================
# IMf = Inductive Miner (infrequent) — a *more flexible* configuration.
# It captures more behavioral detail (higher fitness) at the cost of more
# complex / less generalizable models.

In [ ]:
#---BALANCED-----
# Keeps most traces
# Removes some infrequent paths
# Produces a reasonably interpretable model

# --- NEW: IMf variant specifically for per-cluster discovery ---
def discover_model_for_miner_imf(log):
    """
    IMf version — matches old pipeline behavior.
    """
    tree = inductive_miner.apply(
        log,
        variant=inductive_miner.Variants.IMf,
        parameters={"noise_threshold": 0.2}
    )
    net, im, fm = pt_converter.apply(tree)
    return net, im, fm, tree

In [ ]:
#---STRICT(simpler models, less fitness)-----
# Filters weak paths aggressively
# Great for large noisy logs
# Model may underfit (i.e. lose rare but valid behavior)
#def discover_model_for_miner_imf(log):
    """
    IMf version — matches old pipeline behavior.
    """
    #tree = inductive_miner.apply(
        #log,
        #variant=inductive_miner.Variants.IMf,
        #parameters={
            #"noise_threshold": 0.4,
            #"min_dfg_occurrences": 2
        #}
    #)
    #net, im, fm = pt_converter.apply(tree)
    #return net, im, fm, tree

In [ ]:
#---FLEXIBLE(complex model, higher fitness)-----
# Almost no pruning
# captures most behavior
# Model may overfit(i.e. messy and less generalizable)
#def discover_model_for_miner_imf(log):
    """
    IMf version — matches old pipeline behavior.
    """
    #tree = inductive_miner.apply(
        #log,
        #variant=inductive_miner.Variants.IMf,
        #parameters={
            #"noise_threshold": 0.1,
            #"min_dfg_occurrences": 1
        #}
    #)
    #net, im, fm = pt_converter.apply(tree)
    #return net, im, fm, tree

In [ ]:

# =============================================================
# 1) Per‑Cluster Discovery & Evaluation
# =============================================================

def discover_and_evaluate_per_cluster(
    df: pd.DataFrame,
    filtered: pd.DataFrame,
    cluster_col: str,
    method_name: str = None,
    cluster_metrics_df: pd.DataFrame = None,   # ← renamed
    skip_noise: bool = True,
    noise_label: int = -1,
    visualize: bool = False,
    outdir: str = "cluster_models",
    sample_size: int = None,
):
    """
    Discover and evaluate process models *independently for each cluster*.


    Args:
    df: Original full event log (pandas DataFrame).
    filtered: DataFrame containing case IDs + cluster assignments.
    cluster_col: Column name inside `filtered` identifying cluster labels.
    method_name: Name to insert in the metrics output (defaults to cluster_col).
    cluster_metrics_df: Existing table to append results to.
    skip_noise: If True, skip the noise cluster (e.g., HDBSCAN label -1).
    noise_label: Integer label representing noise.
    visualize: If True, render BPMN for each cluster.
    outdir: Directory for saving models (not used here but reserved).
    sample_size: Max traces to evaluate (None → default = EVAL_SAMPLE_SIZE).


    Returns:
    Updated cluster_metrics_df containing one row per cluster.
    """
    method_name = method_name or cluster_col
    #sample_size = sample_size if sample_size is not None else EVAL_SAMPLE_SIZE
    
    # ----------------------------------------------------------
    # Validate index structure (cases must be index or a column)
    # ----------------------------------------------------------
    if filtered.index.name != "case:concept:name":
        if "case:concept:name" in filtered.columns:
            filtered = filtered.set_index("case:concept:name", drop=True)
        else:
            raise ValueError("`filtered` must have case ids on the index or a 'case:concept:name' column.")

    if cluster_col not in filtered.columns:
        raise ValueError(f"`{cluster_col}` not found in filtered columns: {list(filtered.columns)}")

    # ----------------------------------------------------------
    # Iterate over clusters
    # ----------------------------------------------------------
    for c, case_ids in filtered.groupby(cluster_col).groups.items():
        if skip_noise and c == noise_label:
            continue

        cluster_df = df[df["case:concept:name"].isin(case_ids)].copy()
        if cluster_df.empty:
            continue
        
        # Extract cluster-specific subset of df
        cluster_df = dataframe_utils.convert_timestamp_columns_in_df(cluster_df)
        
        # Standard timestamp normalization for PM4Py
        cluster_df = cluster_df.sort_values(
            ["case:concept:name","time:timestamp"],
            ignore_index=True
        )
        # Mapping for PM4Py conversion
        params = {
            "case_id":       "case:concept:name",
            "activity_key":  "concept:name",
            "timestamp_key": "time:timestamp",
        }
        
        

        log = log_converter.apply(cluster_df, variant=log_converter.Variants.TO_EVENT_LOG, parameters=params)

        # ------------------------------------------------------
        # Discover model (IMf variant for detailed structure)
        # ------------------------------------------------------
        net, im, fm, tree = discover_model_for_miner_imf(log)

        eval_log = log
        
        # ------------------------------------------------------
        # Conformance metrics
        # ------------------------------------------------------
        fit_res  = fitness_evaluator.apply(eval_log, net, im, fm, variant=FITNESS_VARIANT)
        fitness  = fit_res.get("log_fitness", None)

        precision = precision_evaluator.apply(eval_log, net, im, fm, variant=PRECISION_VARIANT)
        
        if precision is not None and fitness is not None and (precision+fitness) > 0:
            fscore    = (2 * precision * fitness / (precision + fitness))  
        else: 
            fscore = 0.0

        variability = compute_variability_ratio(log)
        
        # ------------------------------------------------------
        # Append metrics row
        # ------------------------------------------------------

        row_dict = {
            
            "Method":           method_name,
            "Cluster":          f"Cluster {c}",
            "NumTraces":        len(log),
            "Precision":        float(precision),
            "Fitness":          float(fitness),
            "FScore":           float(fscore),
            "VariabilityRatio": float(variability),
            "Miner":            "inductive"
        }

        # ← saving to the cluster-level dataframe
        if (cluster_metrics_df is None) or (len(cluster_metrics_df) == 0):
            cluster_metrics_df = pd.DataFrame([row_dict])
        else:
            cluster_metrics_df = pd.concat(
                [cluster_metrics_df, pd.DataFrame([row_dict])],
                ignore_index=True
            )

        print(
            f"✅ {cluster_col} / Cluster {c} (IMf) → "
            f"Prec {precision:.3f} | Fit {fitness:.3f} | F1 {fscore:.3f} | Var {variability:.3f} | "
            f"Traces {len(log)}"
        )
        
        # ------------------------------------------------------
        # Optional visualization
        # ------------------------------------------------------

        if visualize:
            #print(f"Rendering PN for cluster {c}...")
            #gviz = pn_visualizer.apply(net, im, fm)
            #pn_visualizer.view(gviz)
            print(f"Rendering BPMN for cluster {c}...")
            bpmn_graph = process_tree_converter.apply(
                            tree,
                            variant=process_tree_converter.Variants.TO_BPMN
            )
            gviz = bpmn_visualizer.apply(bpmn_graph)
            bpmn_visualizer.view(gviz)

    return cluster_metrics_df    # important for downstream analysis


### TFIDF Calls

In [ ]:
# Kmeans + TFIDF  (FIRST)
cluster_metrics_df = discover_and_evaluate_per_cluster(
    df=df,
    filtered=filtered,
    cluster_col="cluster_kmeans_tfidf_svd",
    method_name="KMeans-TFIDF",
    cluster_metrics_df=None,          # ← When set to none, it resets the dataframe cluster_metrics_df
    skip_noise=True,
    visualize=True
)

# SOM + TFIDF
cluster_metrics_df = discover_and_evaluate_per_cluster(
    df=df,
    filtered=filtered,
    cluster_col="cluster_som_tfidf_svd",
    method_name="SOM-TFIDF",
    cluster_metrics_df=cluster_metrics_df,  # << append, not None
    skip_noise=True,
    visualize=True
)

# HDBSCAN + TFIDF
cluster_metrics_df = discover_and_evaluate_per_cluster(
    df=df,
    filtered=filtered,
    cluster_col="cluster_hdbscan_tfidf_svd",
    method_name="HDBSCAN-TFIDF",
    cluster_metrics_df=cluster_metrics_df,  # << append, not None
    skip_noise=True,
    noise_label=-1,
    visualize=True
)

display(cluster_metrics_df)


### Doc2Vec Calls

In [ ]:
# KMeans + Doc2Vec
cluster_metrics_df = discover_and_evaluate_per_cluster(
    df=df,
    filtered=filtered,
    cluster_col="cluster_kmeans_doc2vec",
    method_name="KMeans-Doc2Vec",
    cluster_metrics_df=cluster_metrics_df,     # first call in this block
    skip_noise=True,
    visualize=True
)

# SOM + Doc2Vec
cluster_metrics_df = discover_and_evaluate_per_cluster(
    df=df,
    filtered=filtered,
    cluster_col="cluster_som_doc2vec",
    method_name="SOM-Doc2Vec",
    cluster_metrics_df=cluster_metrics_df,   # append to same df
    skip_noise=True,
    visualize=True
)

# HDBSCAN + Doc2Vec
cluster_metrics_df = discover_and_evaluate_per_cluster(
    df=df,
    filtered=filtered,
    cluster_col="cluster_hdbscan_doc2vec",
    method_name="HDBSCAN-Doc2Vec",
    cluster_metrics_df=cluster_metrics_df,   # append again
    skip_noise=True,
    noise_label=-1,
    visualize=True
)


display(cluster_metrics_df)


In [ ]:
cluster_metrics_df.to_csv("cluster_metrics_results.csv", index=False)


In [ ]:
tfidf_only = cluster_metrics_df[cluster_metrics_df["Method"].str.contains("TFIDF")]

# Weighted averages of conformance metrics
# Should a cluster with 5 cases influence overall analysis as much as a cluster with 400 cases? (in this case, no)
weights = tfidf_only["NumTraces"]

weighted_avg_tfidf = (tfidf_only[["Precision", "Fitness", "FScore"]]
                      .multiply(weights, axis=0)
                      .sum() / weights.sum())

weighted_avg_tfidf


In [ ]:
# Filter only TFIDF-based methods
tfidf_only = cluster_metrics_df[cluster_metrics_df["Method"].str.contains("TFIDF")]

# Weighted averages per method (TFIDF only)
method_weighted_tfidf = (
    tfidf_only
    .groupby("Method")
    .apply(lambda g: (
        (g[["Precision","Fitness","FScore"]] * g["NumTraces"].values[:,None]).sum()
        / g["NumTraces"].sum()
    ))
)

method_weighted_tfidf
